In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import seaborn as sns
from matplotlib import rcParams

from sklearn.ensemble import RandomForestClassifier

# Complete example using RandomForestClassifier

https://github.com/benhamner/ExpediaPersonalizedSortCompetition/

In [ ]:
df = pd.read_csv('../../training_set_VU_DM_2014.csv').fillna(value=0)

In [ ]:
# remove features not available in test set
feature_names = list(df.columns)
feature_names.remove("click_bool")
feature_names.remove("booking_bool")
feature_names.remove("gross_bookings_usd")
feature_names.remove("date_time")
feature_names.remove("position")

In [ ]:
# split into features and target
features = df[feature_names].values
target = df["booking_bool"].values

In [ ]:
# train model
print("Training the Classifier")
classifier = RandomForestClassifier(n_estimators=50, 
                                        verbose=2,
                                        n_jobs=4,
                                        min_samples_split=10,
                                        random_state=1)
classifier.fit(features, target)

In [ ]:
# # save model
# filename = 'model.sav'
# pickle.dump(classifier, open(filename, 'wb'))

# # load model
# loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
# load test set
test = pd.read_csv('../../test_set_VU_DM_2014.csv').fillna(value=0)

In [ ]:
# obtain feature names and remove useless column date_time
feature_names = list(test.columns)
feature_names.remove("date_time")

# obtain feature values
features = test[feature_names].values

# predict using trained model
print("Making predictions")
predictions = classifier.predict_proba(features)[:,1]
predictions = list(-1.0*predictions)
recommendations = zip(test["srch_id"], test["prop_id"], predictions)

In [ ]:
# sort based on recommendation value and write sorted prediction to file

from operator import itemgetter
import csv

rows = [(srch_id, prop_id)
        for srch_id, prop_id, rank_float
        in sorted(recommendations, key=itemgetter(0,2))]
writer = csv.writer(open("predict.csv", "w"), lineterminator="\n")
writer.writerow(("SearchId", "PropertyId"))
writer.writerows(rows)